## Base model
We will use other pretrained models combined together to generate new hair styles

In [ ]:
from PIL import Image, ImageDraw
from diffusers import StableDiffusionInpaintPipeline
import matplotlib.pyplot as plt
import torch

# Load the original image
image = Image.open("output.png").convert("RGB")

# Create a mask image (same size as the input image, filled with black)
mask_image = Image.new("L", image.size, 0)  # "L" mode is grayscale (black/white)

# Define a region to inpaint (e.g., a rectangle)
draw = ImageDraw.Draw(mask_image)
draw.rectangle([50, 50, 500, 500], fill=255)  # White region (inpaint this part)
mask_image.save("mask.png")

# Load the Stable Diffusion inpainting model
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16
)

# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)

# Run inpainting
image = pipe(
    prompt="fade the man up, give him a mohawk",
    image=image,
    mask_image=mask_image
).images[0]

# Save the new image
image.save("new_image.png")

# Display the result
plt.imshow(image)
plt.axis("off")
plt.show()

I add the image segmentor so now we can input any image and prompt and new hair will generate

In [ ]:
from transformers import pipeline
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2  # For mask dilation

# 1. Load Models
segmentation_model = pipeline("image-segmentation", model="jonathandinu/face-parsing")

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16
)

# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)

# 2. Image Input
image = Image.open("images/headshot1.jpg").convert("RGB")

# 3. Hair Segmentation
segmentation_results = segmentation_model(image)
hair_mask = None
for result in segmentation_results:
    if result["label"].lower() == "hair":
        hair_mask = result["mask"]
        break

if hair_mask is None:
    raise ValueError("Hair segmentation not found in the image.")

# Convert hair_mask to a NumPy array if it's a PIL Image
if isinstance(hair_mask, Image.Image):
    hair_mask = np.array(hair_mask)

# Ensure binary format
hair_mask = (hair_mask > 0).astype(np.uint8)

# 4. Expand Mask by X pixels (dilate)
X = 10  # <-- You can change this to any number of pixels to expand
kernel = np.ones((2 * X + 1, 2 * X + 1), np.uint8)
dilated_mask = cv2.dilate(hair_mask, kernel, iterations=1)

# Convert to PIL Image for inpainting
mask_image = Image.fromarray((dilated_mask * 255).astype(np.uint8)).resize(image.size)

# 5. Stable Diffusion Inpainting
prompt = "A green afro haircut"
result_image = pipe(
    prompt=prompt,
    image=image,
    mask_image=mask_image
).images[0]

# 6. Display Input, Mask and Output Images using matplotlib
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
axs[0].imshow(image)
axs[0].set_title("Input Image")
axs[0].axis("off")

axs[1].imshow(mask_image, cmap="gray")
axs[1].set_title(f"Expanded Hair Mask (+{X}px)")
axs[1].axis("off")

axs[2].imshow(result_image)
axs[2].set_title("Output Image")
axs[2].axis("off")

plt.tight_layout()
plt.show()
